In [ ]:
#from skimage.measure import label

In [ ]:
def ellipse_pixels(imarray, center, semi_axes, rotation, image_shape):
    """
    Generate the pixel coordinates that lie inside an ellipse.

    Parameters:
    - center: Tuple (x, y) representing the center of the ellipse.
    - semi_axes: Tuple (semi_major_axis, semi_minor_axis) representing the lengths of the ellipse's axes.
    - rotation: Rotation angle of the ellipse in radians.
    - image_shape: Shape of the image (height, width) to constrain the ellipse.

    Returns:
    - A 2D array of pixel coordinates (row, column) that lie inside the ellipse.
    """
    # Create a grid of x and y coordinates for the image
    y, x = np.meshgrid(np.arange(imarray.shape[1]), np.arange(imarray.shape[0]), indexing='xy')
    
    # Compute the cosine and sine of the rotation angle
    cos_theta = np.cos(rotation)
    sin_theta = np.sin(rotation)
    
    # Rotate the x and y coordinates to align with the ellipse's axes
    x_rot = cos_theta * (x - center[0]) + sin_theta * (y - center[1])
    y_rot = -sin_theta * (x - center[0]) + cos_theta * (y - center[1])
source
    """
    Generate the pixel coordinates that lie inside an ellipse.

    Parameters:
    - center: Tuple (x, y) representing the center of the ellipse.
    - semi_axes: Tuple (semi_major_axis, semi_minor_axis) representing the lengths of the ellipse's axes.
    - rotation: Rotation angle of the ellipse in radians.
    - image_shape: Shape of the image (height, width) to constrain the ellipse.

    Returns:
    - A 2D array of pixel coordinates (row, column) that lie inside the ellipse.
    """
    # Create a grid of x and y coordinates for the image
    y, x = np.meshgrid(np.arange(imarray.shape[1]), np.arange(imarray.shape[0]), indexing='xy')
    
    # Compute the cosine and sine of the rotation angle
    cos_theta = np.cos(rotation)
    sin_theta = np.sin(rotation)
    
    # Rotate the x and y coordinates to align with the ellipse's axes
    x_rot = cos_theta * (x - center[0]) + sin_theta * (y - center[1])
    y_rot = -sin_theta * (x - center[0]) + cos_theta * (y - center[1])

    # Create a mask for pixels that lie inside the ellipse
    mask = (x_rot / semi_axes[0])**2 + (y_rot / semi_axes[1])**2 <= 1

    # Enable interactive mode for Napari (if needed)
    settings.application.ipy_interactive = True

    # Return the coordinates of the pixels inside the ellipse
    return np.column_stack(np.where(mask))

def rgb_to_hsv(rgb_array):
    """
    Convert an array of RGB values to HSV (Hue, Saturation, Value).

    Parameters:
    - rgb_array: A 2D array where each row is an RGB triplet (R, G, B).

    Returns:
    - A 2D array where each row is an HSV triplet (H, S, V).
      H is in degrees (0-360), S and V are percentages (0-100).
    """
    hsv_list = []
    for rgb in rgb_array:
        r, g, b = rgb  # Extract the RGB components
        h, s, v = colorsys.rgb_to_hsv(r, g, b)  # Convert RGB to HSV
        hsv_list.append([h * 360, s * 100, v * 100])  # Scale H to degrees, S and V to percentages
    return np.array(hsv_list)

def csv_to_matrix(file_path):
    """
    Read a CSV file and convert its contents into a matrix.

    Parameters:
    - file_path: Path to the CSV file.

    Returns:
    - A 2D list where each row corresponds to a row in the CSV file.
    """
    matrix = []
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            # Convert numeric values to integers, leave others as strings
            matrix.append([int(value) if value.replace('.', '', 1).isdigit() else value for value in row])
    return matrix

def classify_points(cloud_points, test_points, bandwidth=1.0):
    """
    Classify test points based on their similarity to a cloud of points using Kernel Density Estimation (KDE).

    Parameters:
    - cloud_points: A 2D array of points representing the training data.
    - test_points: A 2D array of points to classify.
    - bandwidth: Bandwidth parameter for the KDE (controls smoothness).

    Returns:
    - A 1D array of probabilities for each test point.
    """
    # Fit a Kernel Density Estimator to the cloud points
    kde = KernelDensity(bandwidth=bandwidth, kernel='gaussian')
    kde.fit(cloud_points)
    
    # Evaluate the probability density for each test point
    log_density = kde.score_samples(test_points)
    
    # Convert log-density to probabilities
    probabilities = np.exp(log_density)
    return probabilities

def calibrate_image_resolution(image: np.ndarray):
    """
    Opens a Napari viewer for interactive calibration.
    The user draws a line over the scalebar using the Shapes layer (Line)
    and presses 'q' to confirm. After the viewer closes the function will
    ask for the real-world scalebar length (in µm) and return the pixel
    resolution in µm/pixel.

    This function will loop and offer a retry if no line was confirmed
    (to avoid the need to re-run the calling cell).
    """

    while True:
        viewer = napari.Viewer()
        viewer.add_image(image, name='image')
        shapes_layer = viewer.add_shapes(name='scalebar', shape_type='line')

        # Provide clear on-screen instructions before opening Napari
        settings.application.ipy_interactive = False
        print("\n🟢 Napari opened: draw a line over the scalebar using the Shapes tool (Line).")
        print("When finished press 'q' while the Napari window is focused to confirm the measurement.")

        result = {}

        @viewer.bind_key('q')
        def _on_confirm(event=None):
            """Callback triggered when 'q' is pressed."""
            if len(shapes_layer.data) == 0:
                print("⚠️ No line found! Please draw one first and press 'q' again.")
                return
            line_coords = shapes_layer.data[0]
            p1, p2 = line_coords
            pixel_length = np.linalg.norm(p2 - p1)
            print(f"Measured scalebar length: {pixel_length:.2f} pixels")
            result['pixel_length'] = pixel_length
            viewer.close()

        napari.run()  # Waits until viewer is closed

        # If user confirmed a line, proceed to ask for real-world length
        if 'pixel_length' in result:
            try:
                real_length_um = float(input("Enter the real-world scalebar length (in µm): "))
            except Exception:
                raise ValueError("Invalid real-world length entered.")
            resolution_um_per_px = real_length_um / result['pixel_length']
            print(f"\n✅ Pixel resolution: {resolution_um_per_px:.4f} µm/pixel")
            return resolution_um_per_px

        # If no line was confirmed, offer a retry to the user
        print("No scalebar line was confirmed in Napari.")
        retry = input("Retry calibration? (y/n): " ).strip().lower()
        if retry != 'y':
            raise ValueError("Calibration aborted by user. No line was confirmed.")

In [62]:
settings.application.ipy_interactive = True

# Visualize original, mask, and ROI in Napari
viewer_0 = napari.Viewer()

viewer_0.add_image(imarray, name='Original', 
                colormap='grey', blending='additive')
viewer_0.add_image(mask_filled, name='Violet', 
                colormap='green', blending='additive')
viewer_0.add_image(ROI_image, name='ROI', 
                colormap='grey', blending='additive')

<Image layer 'ROI' at 0x7279ee626e90>

After you run the next section, a new Napari window will appear with highlighted in green all the nuclei in the ROI.

In [63]:
# Enable interactive mode for Napari
settings.application.ipy_interactive = True

# Create a copy of the ROI image for processing
imarray0 = ROI_image.copy()

# Extract the red, green, and blue channels from the ROI image
imR = imarray0[:, :, 0]
imG = imarray0[:, :, 1]
imB = imarray0[:, :, 2]

# Convert the training points from RGB to HSV
yes_points_hsv = rgb_to_hsv(yes_points)

# Scale the HSV training points for better performance in the SVM
scaler = StandardScaler()
yes_points_scaled = scaler.fit_transform(yes_points_hsv)

# Set up the One-Class SVM for anomaly detection
sensitivity = 0.8  # Sensitivity level (e.g., 10% of points allowed as outliers)
clf = OneClassSVM(kernel='rbf', nu=sensitivity, gamma='scale')
clf.fit(yes_points_scaled)

# Initialize an empty array for the violet channel
imV = np.zeros(np.shape(imR))

# Initialize a counter for progress tracking
tval = 0

# Iterate through the image to classify each pixel
for i in range(step, np.shape(imV)[0]):
    for j in range(step, np.shape(imV)[1]):
        tval += 1
        # Process only pixels within the mask
        if mask_filled[i, j] > 0:
            # Extract RGB values of the current pixel
            pR = imR[i, j]
            pG = imG[i, j]
            pB = imB[i, j]
            X = np.array([[pR, pG, pB]])

            # Convert the pixel to HSV and scale it
            X_hsv = rgb_to_hsv(X)
            X_scaled = scaler.transform(X_hsv)

            # Classify the pixel using the KDE-based classifier
            predictions = classify_points(yes_points, X, bandwidth=20.0)

            # Assign the prediction value to the violet channel
            if predictions > 0.0:
                imV[i, j] = predictions[0]
            else:
                imV[i, j] = 0
        else:
            j += step  # Skip to the next step if outside the mask

        # Display progress at regular intervals
        if (100.0 * tval / (np.shape(imV)[0] * np.shape(imV)[1]) % 1.0 == 0.0):
            clear_output(wait=True)
            print('PROGRESS ' + str(100.0 * tval / (np.shape(imV)[0] * np.shape(imV)[1])) + ' %')

# Visualize the results in Napari
viewer_0 = napari.Viewer()

# Add the image to the viewer
viewer_0.add_image(imarray, name='Original', 
                colormap='grey', blending='additive')

# Add the violet channel (classification results) to the viewer
viewer_0.add_image(imV, name='Violet', 
                   colormap='green', blending='additive')


PROGRESS 75.0 %


<Image layer 'Violet' at 0x7279ec1d9ff0>

#### Remove noise and big regions

In [64]:
# Visualization
viewer_full = napari.Viewer()
viewer_full.add_image(imarray, name='Original', colormap='grey', blending='additive')
viewer_full.add_image(imV, name='Violet', colormap='green', blending='additive')

<Image layer 'Violet' at 0x7279e420faf0>

In [65]:
#🔹 Step 1: Thresholding: switch from grayscale to binary scale (different brightness to either 0 for background or 1 for nuclei)
im_in=imV

# Get all nonzero values in imV (ignoring background)
valid_pixels = imV[imV > 0]  

# Compute the percentile threshold only for these valid pixels
if len(valid_pixels) > 0:  # Ensure there are nonzero pixels
    threshold_value = np.percentile(valid_pixels, 95.0)
    strong_th_value = np.percentile(valid_pixels, 98.0)
    binary_mask = (imV >= threshold_value)  # Keep only the top 5% brightest pixels
    strong_b_mask = (imV >= strong_th_value)  # Keep only the top 5% brightest pixels
else:
    binary_mask = np.zeros_like(imV)  # If no valid pixels, return empty mask
    
viewer_full.add_image(binary_mask.astype(float), name="Thresholded", colormap="blue", blending="additive")
napari.run()
im_out=binary_mask.copy()

In [66]:
# Step 2: Remove Noise & Artifacts
im_in=im_out
binary_mask=im_in

# Remove small objects and close gaps
binary_mask2 = remove_small_objects(binary_mask, min_size=5)  
binary_mask2 = closing(binary_mask2, square(3))  # Fills small holes and connects nearby white regions.

strong_b_mask2 = remove_small_objects(strong_b_mask, min_size=5)  
strong_b_mask2 = closing(strong_b_mask2, square(3))  # Fills small holes and connects nearby white regions.

viewer_full.add_image(binary_mask2.astype(float), name="Noise and art. removed", colormap="blue", blending="additive")
napari.run()
im_out=binary_mask2.copy()

In [67]:
# Step 3: Remove Artifacts Based on Size and Roundness
im_in=im_out
binary_mask2=im_in

min_roundness = 0.75

# Define the minimum and maximum size for nuclei to be considered valid
min_nucleus_diam = 3  # Minimum diameter of a nucleus (um)
max_nucleus_diam = 20  # Maximum diameter of a nucleus (um)

min_nucleus_diam_res = min_nucleus_diam/((r_X+r_Y)/2)
max_nucleus_diam_res = max_nucleus_diam/((r_X+r_Y)/2)

min_nucleus_size=(min_nucleus_diam/2)*(min_nucleus_diam/2)*np.pi
max_nucleus_size=(max_nucleus_diam/2)*(max_nucleus_diam/2)*np.pi

# Initialize an empty mask to store the filtered regions
filtered_mask = np.zeros_like(binary_mask2)
large_nuclei_mask = filtered_mask.copy()

# Iterate through each connected region in the labeled binary mask
for region in regionprops(label(binary_mask2)):
    # Check if the region's area falls within the valid size range
    if min_nucleus_size <= region.area <= max_nucleus_size:
        if region.perimeter > 0:
            # Compute roundness (circularity)
            roundness = 4 * np.pi * region.area / (region.perimeter ** 2)

            # Keep only regions that are sufficiently round
            if roundness >= min_roundness:
                # Add the region to the filtered mask
                filtered_mask[label(binary_mask2) == region.label] = 1

# Iterate through each connected region in the labeled binary mask
for region in regionprops(label(strong_b_mask2)):
    # Check if the region's area falls within the valid size range
    if min_nucleus_size <= region.area <= max_nucleus_size:
        if region.perimeter > 0:
            # Compute roundness (circularity)
            roundness = 4 * np.pi * region.area / (region.perimeter ** 2)

            # Keep only regions that are sufficiently round
            if roundness >= min_roundness:
                # Add the region to the filtered mask
                filtered_mask[label(strong_b_mask2) == region.label] = 1

viewer_full.add_image(filtered_mask.astype(float), name="Size and roundness threshold", colormap="blue", blending="additive")
napari.run()
im_out=filtered_mask.copy()

In [68]:
# #🔹 Step 4: Filter by Size and Roundness
# im_in = im_out
# segmented_nuclei = im_in

# # Define the minimum roundness value (1.0 = perfect circle)
# min_roundness = 0.75  # adjust as needed (0.7–0.85 works well for nearly round nuclei)

# # Initialize an empty mask to store the filtered segments
# filtered_segments = np.zeros_like(segmented_nuclei)

# # Initialize a counter for assigning new labels to filtered segments
# k = 1

# # Iterate through each connected region in the segmented nuclei
# for region in regionprops(segmented_nuclei):
#     if region.perimeter > 0:
#         # Compute roundness (circularity)
#         roundness = 4 * np.pi * region.area / (region.perimeter ** 2)

#         # Keep only regions that are sufficiently round
#         if roundness >= min_roundness:
#             filtered_segments[segmented_nuclei == region.label] = k
#             k += 1

# viewer_full.add_labels(filtered_segments, name="Filtered by Roundness")
# napari.run()
# im_out = filtered_segments.copy()

In [69]:
# Step 4: Watershed for Better Separation
im_in=im_out
filtered_mask=im_in

# Compute the distance transform of the filtered mask
distance = distance_transform_edt(filtered_mask)

# Create markers for the watershed algorithm based on the distance transform
# Markers are created where the distance is greater than a small fraction of the maximum distance
markers = label(distance > 0.0001 * distance.max())

# Apply the watershed algorithm to segment nuclei
# The negative distance is used to ensure that the watershed grows from the markers
segmented_nuclei = watershed(-distance, markers, mask=filtered_mask)

viewer_full.add_labels(segmented_nuclei, name="Segmented Nuclei")
napari.run()
im_out=segmented_nuclei.copy()

### Output 
Creates a .tiff file with multiple pages. P1 is the original image, P2 is the ROI chosen as the tissue, P3 is the detected nuclei.

In [70]:
filtered_segments=segmented_nuclei

In [ ]:
# Convert the original image array to an RGB image
rgb_im = Image.fromarray(imarray.astype(np.uint8), mode="RGB")

# Convert the ROI image array to an RGB image
rgb0_im = Image.fromarray(imarray0.astype(np.uint8), mode="RGB")

# Initialize an empty RGB array for the filtered segments
filtered_segments_rgb = np.zeros((np.shape(filtered_segments)[0], np.shape(filtered_segments)[1], 3))

# Generate random colors for each segment label
cmd = np.random.rand(np.max(filtered_segments) + 1, 3)
cmd[0, :] = [0.0, 0.0, 0.0]  # Ensure the background (label 0) is black

# Assign colors to each pixel based on its segment label
for i in range(1, np.shape(filtered_segments)[0]):
    for j in range(1, np.shape(filtered_segments)[1]):
        filtered_segments_rgb[i, j, 0] = int(cmd[filtered_segments[i, j], 0] * 255.0)  # Red channel
        filtered_segments_rgb[i, j, 1] = int(cmd[filtered_segments[i, j], 1] * 255.0)  # Green channel
        filtered_segments_rgb[i, j, 2] = int(cmd[filtered_segments[i, j], 2] * 255.0)  # Blue channel

# Convert the RGB array to uint8 format
filtered_segments_rgb = filtered_segments_rgb.astype('uint8')

# Convert the filtered segments RGB array to an image
b_im = Image.fromarray(filtered_segments_rgb, mode="RGB")

# Save the original image, ROI image, and filtered segments image in a single multi-page TIFF file
output_path ='./Output_files/' + tiff_stem + "_output.tiff"
rgb_im.save(output_path, save_all=True, append_images=[rgb0_im, b_im])

# Print the output file path
print(f"TIFF file saved at: {output_path}")

TIFF file saved at: pAC024 D28 uM SafO slide7_output.tiff


### QUANTIFICATION

In [72]:
#from scipy.ndimage import label, center_of_mass

# Get unique labels from the filtered segments (excluding the background label 0)
labels = np.unique(filtered_segments)
labels = labels[labels != 0]

# Print the total number of nuclei detected
print('TOTAL NUCLEI ' + str(len(labels+1)))

# Compute the centroids (barycenters) of each nucleus
barycenters = {l: center_of_mass(filtered_segments == l) for l in labels}

# Compute the area of each nucleus in square micrometers
areas = {l: np.sum(filtered_segments == l) * r_X * r_Y for l in labels}

# Convert barycenters to a dictionary of coordinates in micrometers
barycenter_coords = {k: (round(v[0], 2) * r_X, round(v[1], 2) * r_Y) for k, v in barycenters.items()}

# Calculate the total area of the image in square micrometers
fullA = np.prod(np.shape(mask_image)) * r_X * r_Y

# Calculate the total area of the ROI in square micrometers
roiA = np.sum(mask_image) * r_X * r_Y

# Print the total area of the image and the ROI
print("TOTAL AREA IMAGE %.2e um2" % fullA)
print("TOTAL AREA ROI %.2e um2" % roiA)

# Calculate the concentration of cells in the ROI (cells per square micrometer)
roiCON = (len(labels+1)) / roiA
roiCONmm=roiCON*1e6

# Print the cell concentration in the ROI
print("CELL CONCENTRATION in ROI %.2e cells/mm2" % roiCONmm)

TOTAL NUCLEI 2166
TOTAL AREA IMAGE 3.88e+07 um2
TOTAL AREA ROI 5.83e+06 um2
CELL CONCENTRATION in ROI 3.71e+02 cells/mm2


In [ ]:
# Create a new Excel workbook
workbook = xlsxwriter.Workbook('./Output_files/' + tiff_stem + '.xlsx')

## FORMATS
# Define a format for headers (bold text with yellow background)
header = workbook.add_format({'bold': True})
header.set_bg_color('yellow')

# Define a format for floating-point numbers
floats = workbook.add_format({'num_format': '0.00'})

# Define a format for exponential numbers
exp = workbook.add_format()
exp.set_num_format(11)

## CELLS
# Create a worksheet for cell data
worksheet_cell = workbook.add_worksheet('Cells')

# HEADER
# Write the header row for the 'Cells' worksheet
worksheet_cell.write_row('A1:E1', ['#ID', 'X [um]', 'Y [um]', 'Area Nuclei [um2]'], header)

# CONTENT
# Write the data for each nucleus
for row, value in enumerate(labels):
    worksheet_cell.write(row + 1, 0, value)  # Write nucleus ID
    worksheet_cell.write(row + 1, 1, barycenter_coords[value][0], floats)  # Write X coordinate
    worksheet_cell.write(row + 1, 2, barycenter_coords[value][1], floats)  # Write Y coordinate
    worksheet_cell.write(row + 1, 3, areas[value], floats)  # Write area of the nucleus
    clear_output(wait=True)  # Clear the output to show progress
    print('NUCLEI ' + str(row + 1) + ' / ' + str(len(labels + 1)))  # Print progress

## ROI SHEET
# Create a worksheet for ROI data
worksheet_ROI = workbook.add_worksheet('ROI')

# HEADER
# Write the header row for the 'ROI' worksheet
worksheet_ROI.write_row('A1:E1', ['# NUCLEI', 'TOT AREA [um2]', 'ROI AREA [um2]', 'CONC NUCLEI in ROI [cells/mm2]'], header)

# CONTENT
# Write the ROI data
worksheet_ROI.write(1, 0, len(labels + 1))  # Write the total number of nuclei
worksheet_ROI.write(1, 1, fullA, exp)  # Write the total area of the image
worksheet_ROI.write(1, 2, roiA, exp)  # Write the total area of the ROI
worksheet_ROI.write(1, 3, roiCONmm, exp)  # Write the cell concentration in the ROI

# Close the workbook to save the file
workbook.close()